In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path=".env", override=True)

In [ ]:
## Tools

from langchain_openai import ChatOpenAI
from pydantic import BaseModel,Field
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
import requests
import json
from langchain.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException

llm = ChatOpenAI(model="gpt-4o")


## Pydantic Class

class Employee(BaseModel):
    """Represents a single employee with name, age, and salary."""
    full_name: str = Field(description="The full name of the employee.")
    basic_salary: int = Field(description="The yearly base salary of the employee.")
    earn_leaves: float = Field(description="The total accumulated earn leaves of the employee.")
    sick_leaves: float = Field(description="The total accumulated sick leaves the employee.")



@tool
def fetch_emp_data(username:str, password :str) -> dict:
    """
    Fetches employee data from an external API.

    Parameters:
        username (str) : An username of an employee. This can be an email id as well
        password (str) : A passphrase used to authenticate an employee

    Returns:
        dict: A dictionary with employee details.
    """
    payload = {
        "username": username.lower(),
        "password": password
    }
    base_url = "http://127.0.0.1:5000/find_details"
    
    print(f"--- Calling external API to fetch leave data for username: {username} ---")
    
    try:
        response = requests.post(base_url,data=payload)
        data = response.json()
        print ("Response from API : ",data)
        parser = PydanticOutputParser(pydantic_object=Employee)
        structured_data = parser.parse(json.dumps(data))
        print("Structured Response from API: ",structured_data)
        return data
    except requests.exceptions.ConnectionError as e:
        print(f"\nError: Could not connect to the Flask server.")
        print(f"Error details: {e}")
    except requests.exceptions.HTTPError as e:
        print(f"\nError: HTTP request failed with status code {response.status_code}")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")



In [ ]:

# Create a function that can process the output response to a pydentic object 

from langchain.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END , MessagesState
import json

tools = [fetch_emp_data]
tool_node = ToolNode(tools)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Node 1
def tool_calling_llm(state: MessagesState):
    """
    Node to call a tool from the list of tools
    """
    print("executing tool_calling_llm")
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


In [ ]:
# Define the graph
from langgraph.graph import StateGraph, START, END , MessagesState
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage


workflow = StateGraph(MessagesState)
workflow.add_node("tool_calling_llm", tool_calling_llm)
workflow.add_node("tools", tool_node)


workflow.add_edge(START, "tool_calling_llm")
workflow.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
)
workflow.add_edge("tools", END)

graph = workflow.compile()
ascii_graph = graph.get_graph().draw_ascii()
print(ascii_graph)  


# Main chat loop
while True:
    print("\nYou are talking to an employee leave assistant system. \nTo retrieve information please authenticate yourself with your username and password.\n")
    user_input = input("You: ")
    # Check the last message to see if the chat ended by user's command
    if user_input.lower().strip() == "end chat":
        print("AI: Chat ended. Goodbye!")
        break
    else:
        messages = [HumanMessage(content=user_input)]
        messages = graph.invoke({"messages":messages})
        
    for msg in messages['messages']:    
        msg.pretty_print()


